In [ ]:
"""
Python reimplementation of trace_data_analyses_osf.R using scikit-learn and xgboost

- Loads trace_data_study_data_for_modeling.csv
- Drops the extra index column X1 if present
- Computes optional Cronbach's alpha for conscientiousness items (C1, C2r, C3, C4r)
- Prints basic demographic distributions (if columns exist)
- Defines trace features as columns from the 14th onward (index 13), mirroring the R script
- Applies zero-variance and high-correlation filters to trace features
- Trains 5 ML models for g_composite:
    ElasticNet, Random Forest, Gradient Boosting, SVM RBF, XGBoost
- Evaluates with 10-fold CV and a 75/25 train-test split
- Trains analogous models for conscientiousness (if column exists)
- Performs incremental GPA prediction:
    z(GPA) ~ z(game_composite), z(GPA) ~ z(predicted), and z(GPA) ~ both
"""

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.base import clone

import xgboost as xgb
import statsmodels.api as sm
from scipy.stats import zscore, pearsonr

RANDOM_STATE = 180
TEST_SIZE = 0.25  # 75 percent train, 25 percent test


def remove_zero_variance_features(X: pd.DataFrame) -> pd.DataFrame:
    """Remove columns with zero variance."""
    variances = X.var(axis=0)
    keep_cols = variances[variances > 0.0].index
    return X[keep_cols]


def remove_highly_correlated_features(X: pd.DataFrame, threshold: float = 0.90) -> pd.DataFrame:
    """
    Remove one of each pair of features whose absolute correlation exceeds 'threshold'
    Rough analogue of caret::preProcess(..., method = "corr") in R.
    """
    corr_matrix = X.corr().abs()
    upper = corr_matrix.where(
        np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
    )
    to_drop = [
        column for column in upper.columns
        if any(upper[column] > threshold)
    ]
    return X.drop(columns=to_drop)


def cronbach_alpha(df_items: pd.DataFrame) -> float:
    """Compute Cronbach's alpha for a set of item columns."""
    items = df_items.dropna()
    k = items.shape[1]
    if k < 2:
        return np.nan
    item_vars = items.var(axis=0, ddof=1)
    total_var = items.sum(axis=1).var(ddof=1)
    if total_var == 0:
        return np.nan
    alpha = (k / (k - 1)) * (1 - item_vars.sum() / total_var)
    return alpha


def evaluate_model(name, model, X_train, y_train, X_test, y_test, X_full, y_full, cv):
    """
    Fit a model inside a StandardScaler pipeline, run CV, evaluate on test,
    then re-fit on all data to get a final model used for GPA incremental prediction.
    """
    pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("model", model)
    ])

    # Cross-validation on training data
    cv_r2 = cross_val_score(pipe, X_train, y_train, cv=cv, scoring="r2")
    cv_rmse = np.sqrt(-cross_val_score(pipe, X_train, y_train, cv=cv,
                                       scoring="neg_mean_squared_error"))

    # Fit on training, evaluate on held-out test
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    test_r2 = r2_score(y_test, y_pred)

    # Refit on full data for later prediction over the entire dataset
    pipe_full = clone(pipe)
    pipe_full.fit(X_full, y_full)

    print(f"\nModel: {name}")
    print(f"CV R^2:   mean={cv_r2.mean():.3f}, sd={cv_r2.std():.3f}")
    print(f"CV RMSE:  mean={cv_rmse.mean():.3f}, sd={cv_rmse.std():.3f}")
    print(f"Test RMSE={test_rmse:.3f}, Test R^2={test_r2:.3f}")

    return {
        "name": name,
        "cv_r2_mean": cv_r2.mean(),
        "cv_r2_sd": cv_r2.std(),
        "cv_rmse_mean": cv_rmse.mean(),
        "cv_rmse_sd": cv_rmse.std(),
        "test_rmse": test_rmse,
        "test_r2": test_r2,
        "fitted_pipeline": pipe,
        "fitted_full_pipeline": pipe_full
    }


def incremental_prediction_report(label, gpa, game_composite, predicted_scores):
    """
    Replicates the R logic:
    - correlations GPA with game_composite and predicted score
    - linear models:
        M1: z(GPA) ~ z(game_composite)
        M2: z(GPA) ~ z(predicted_scores)
        M3: z(GPA) ~ z(game_composite) + z(predicted_scores)
    - F tests comparing nested models (M1 vs M3, M2 vs M3)
    """
    gpa = pd.Series(gpa)
    game_composite = pd.Series(game_composite)
    predicted_scores = pd.Series(predicted_scores)

    # Standardize
    gpa_z = zscore(gpa, nan_policy='omit')
    game_z = zscore(game_composite, nan_policy='omit')
    pred_z = zscore(predicted_scores, nan_policy='omit')

    # Align valid entries
    mask = np.isfinite(gpa_z) & np.isfinite(game_z) & np.isfinite(pred_z)
    gpa_z = gpa_z[mask]
    game_z = game_z[mask]
    pred_z = pred_z[mask]

    # Model 1
    X1 = sm.add_constant(game_z)
    model1 = sm.OLS(gpa_z, X1).fit()

    # Model 2
    X2 = sm.add_constant(pred_z)
    model2 = sm.OLS(gpa_z, X2).fit()

    # Model 3
    X3 = sm.add_constant(np.column_stack([game_z, pred_z]))
    model3 = sm.OLS(gpa_z, X3).fit()

    # Correlations
    r_game, p_game = pearsonr(game_composite[mask], gpa[mask])
    r_pred, p_pred = pearsonr(predicted_scores[mask], gpa[mask])

    # Incremental F tests
    f12, p12, _ = model1.compare_f_test(model3)
    f23, p23, _ = model2.compare_f_test(model3)

    print(f"\nIncremental GPA prediction using {label}")
    print(f"Correlation GPA ~ game_composite: r={r_game:.3f}, p={p_game:.4g}")
    print(f"Correlation GPA ~ predicted_{label}: r={r_pred:.3f}, p={p_pred:.4g}")

    print("\nModel 1: z(GPA) ~ z(game_composite)")
    print(model1.summary())

    print("\nModel 2: z(GPA) ~ z(predicted)")
    print(model2.summary())

    print("\nModel 3: z(GPA) ~ z(game_composite) + z(predicted)")
    print(model3.summary())

    print("\nF-test: Model 1 vs Model 3 (does predicted add beyond game_composite?)")
    print(f"F={f12:.3f}, p={p12:.4g}")

    print("\nF-test: Model 2 vs Model 3 (does game_composite add beyond predicted?)")
    print(f"F={f23:.3f}, p={p23:.4g}")


def main():
    # Load data
    data = pd.read_csv("trace_data_study_data_for_modeling.csv")

    # Drop extra index column if present
    if "X1" in data.columns:
        data = data.drop(columns=["X1"])

    print("Data shape:", data.shape)
    print("Columns:", list(data.columns))

    # Reliability for conscientiousness items, if present
    consc_items = ["C1", "C2r", "C3", "C4r"]
    if all(col in data.columns for col in consc_items):
        alpha = cronbach_alpha(data[consc_items])
        print(f"Cronbach's alpha for conscientiousness items: {alpha:.3f}")
    else:
        print("Conscientiousness item columns not found, skipping alpha computation")

    # Demographics if present
    if not {"race", "gender", "age", "employed"}.issubset(set(data.columns)):
        print("Some demographic columns missing, skipping detailed demographic stats")
    else:
        print("Race distribution (proportion):")
        print(data["race"].value_counts(normalize=True))

        print("\nGender distribution (proportion):")
        print(data["gender"].value_counts(normalize=True))

        print("\nAge mean and sd (assuming code +17 as in R):")
        age = pd.to_numeric(data["age"], errors="coerce") + 17
        print("Mean age:", age.mean())
        print("SD age:", age.std())

        print("\nEmployment status distribution (proportion):")
        print(data["employed"].value_counts(normalize=True))

    # Checks for required columns
    if "g_composite" not in data.columns:
        raise ValueError("Column 'g_composite' not found in the dataset")

    if "game_composite" not in data.columns:
        raise ValueError("Column 'game_composite' not found in the dataset")

    # Define trace features as columns from 14th onward, mirroring data[,14:ncol(data)] in R
    trace_cols = list(data.columns[13:])
    print("\nUsing the following trace feature columns (from 14th onward):")
    print(trace_cols)

    X_trace = data[trace_cols].copy()
    print("Initial trace feature shape:", X_trace.shape)

    # Preprocessing: zero variance, correlation filter
    X_trace = remove_zero_variance_features(X_trace)
    print("After zero-variance filter:", X_trace.shape)

    X_trace = remove_highly_correlated_features(X_trace, threshold=0.90)
    print("After correlation filter:", X_trace.shape)

    # Target for cognitive ability
    y_g = data["g_composite"].values

    # Shared train-test split indices (used for both g_composite and conscientiousness)
    indices = np.arange(len(data))
    train_idx, test_idx = train_test_split(
        indices,
        test_size=TEST_SIZE,
        random_state=RANDOM_STATE,
        shuffle=True
    )

    X_train_g = X_trace.iloc[train_idx]
    X_test_g = X_trace.iloc[test_idx]
    y_train_g = y_g[train_idx]
    y_test_g = y_g[test_idx]

    # 10-fold CV
    cv = KFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)

    # ---------------------------
    # ML models for g_composite
    # ---------------------------
    results_g = []

    # Elastic Net (glmnet analogue)
    elastic_model = ElasticNetCV(
        l1_ratio=[0.1, 0.5, 0.9],
        alphas=None,
        cv=5,
        random_state=RANDOM_STATE,
        max_iter=10000
    )
    results_g.append(
        evaluate_model("ElasticNet (glmnet analogue)", elastic_model,
                       X_train_g, y_train_g, X_test_g, y_test_g, X_trace, y_g, cv)
    )

    # Random Forest
    rf_model = RandomForestRegressor(
        n_estimators=500,
        max_depth=None,
        random_state=RANDOM_STATE,
        n_jobs=-1
    )
    results_g.append(
        evaluate_model("Random Forest", rf_model,
                       X_train_g, y_train_g, X_test_g, y_test_g, X_trace, y_g, cv)
    )

    # Gradient Boosting (GBM analogue)
    gbm_model = GradientBoostingRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=3,
        subsample=0.8,
        random_state=RANDOM_STATE
    )
    results_g.append(
        evaluate_model("Gradient Boosting (GBM analogue)", gbm_model,
                       X_train_g, y_train_g, X_test_g, y_test_g, X_trace, y_g, cv)
    )

    # SVM with RBF kernel (svmRadial analogue)
    svm_model = SVR(
        kernel="rbf",
        C=10.0,
        gamma="scale",
        epsilon=0.1
    )
    results_g.append(
        evaluate_model("SVM RBF", svm_model,
                       X_train_g, y_train_g, X_test_g, y_test_g, X_trace, y_g, cv)
    )

    # XGBoost regressor
    xgb_model = xgb.XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=4,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        random_state=RANDOM_STATE,
        n_jobs=-1
    )
    results_g.append(
        evaluate_model("XGBoost", xgb_model,
                       X_train_g, y_train_g, X_test_g, y_test_g, X_trace, y_g, cv)
    )

    results_g_df = pd.DataFrame([
        {
            "model": r["name"],
            "cv_r2_mean": r["cv_r2_mean"],
            "cv_rmse_mean": r["cv_rmse_mean"],
            "test_r2": r["test_r2"],
            "test_rmse": r["test_rmse"]
        }
        for r in results_g
    ])
    print("\nOverall comparison for g_composite prediction:")
    print(results_g_df.sort_values(by="test_rmse"))

    # ---------------------------
    # Incremental GPA prediction, g_composite-based models
    # ---------------------------
    gpa = data["gpa"].values if "gpa" in data.columns else None
    game_comp = data["game_composite"].values

    if gpa is not None:
        for r in results_g:
            label_short = r["name"].split()[0].lower()
            pred_all = r["fitted_full_pipeline"].predict(X_trace)
            incremental_prediction_report(
                label=f"g_composite_{label_short}",
                gpa=gpa,
                game_composite=game_comp,
                predicted_scores=pred_all
            )
    else:
        print("\nColumn 'gpa' not found, skipping GPA incremental prediction for g_composite")

    # ---------------------------
    # ML models and incremental GPA prediction for conscientiousness
    # ---------------------------
    if "conscientious" in data.columns:
        y_c = data["conscientious"].values

        X_train_c = X_trace.iloc[train_idx]
        X_test_c = X_trace.iloc[test_idx]
        y_train_c = y_c[train_idx]
        y_test_c = y_c[test_idx]

        results_c = []

        elastic_model_c = ElasticNetCV(
            l1_ratio=[0.1, 0.5, 0.9],
            alphas=None,
            cv=5,
            random_state=RANDOM_STATE,
            max_iter=10000
        )
        results_c.append(
            evaluate_model("ElasticNet (conscientious)", elastic_model_c,
                           X_train_c, y_train_c, X_test_c, y_test_c, X_trace, y_c, cv)
        )

        rf_model_c = RandomForestRegressor(
            n_estimators=500,
            max_depth=None,
            random_state=RANDOM_STATE,
            n_jobs=-1
        )
        results_c.append(
            evaluate_model("Random Forest (conscientious)", rf_model_c,
                           X_train_c, y_train_c, X_test_c, y_test_c, X_trace, y_c, cv)
        )

        gbm_model_c = GradientBoostingRegressor(
            n_estimators=500,
            learning_rate=0.05,
            max_depth=3,
            subsample=0.8,
            random_state=RANDOM_STATE
        )
        results_c.append(
            evaluate_model("Gradient Boosting (conscientious)", gbm_model_c,
                           X_train_c, y_train_c, X_test_c, y_test_c, X_trace, y_c, cv)
        )

        svm_model_c = SVR(
            kernel="rbf",
            C=10.0,
            gamma="scale",
            epsilon=0.1
        )
        results_c.append(
            evaluate_model("SVM RBF (conscientious)", svm_model_c,
                           X_train_c, y_train_c, X_test_c, y_test_c, X_trace, y_c, cv)
        )

        xgb_model_c = xgb.XGBRegressor(
            n_estimators=500,
            learning_rate=0.05,
            max_depth=4,
            subsample=0.8,
            colsample_bytree=0.8,
            objective="reg:squarederror",
            random_state=RANDOM_STATE,
            n_jobs=-1
        )
        results_c.append(
            evaluate_model("XGBoost (conscientious)", xgb_model_c,
                           X_train_c, y_train_c, X_test_c, y_test_c, X_trace, y_c, cv)
        )

        results_c_df = pd.DataFrame([
            {
                "model": r["name"],
                "cv_r2_mean": r["cv_r2_mean"],
                "cv_rmse_mean": r["cv_rmse_mean"],
                "test_r2": r["test_r2"],
                "test_rmse": r["test_rmse"]
            }
            for r in results_c
        ])
        print("\nOverall comparison for conscientiousness prediction:")
        print(results_c_df.sort_values(by="test_rmse"))

        if gpa is not None:
            for r in results_c:
                label_short = r["name"].split()[0].lower()
                pred_all_c = r["fitted_full_pipeline"].predict(X_trace)
                incremental_prediction_report(
                    label=f"conscientious_{label_short}",
                    gpa=gpa,
                    game_composite=game_comp,
                    predicted_scores=pred_all_c
                )
        else:
            print("\nColumn 'gpa' not found, skipping GPA incremental prediction for conscientiousness")
    else:
        print("\nColumn 'conscientious' not found, skipping conscientiousness ML models")


if __name__ == "__main__":
    main()


Data shape: (621, 341)
Columns: ['Unnamed: 0', 'ID', 'gpa', 'g_composite', 'game_composite', 'conscientious', 'age', 'gender', 'race', 'employed', 'C1', 'C2r', 'C3', 'C4r', 'BalloonBlast_count_MouseDown', 'BalloonBlast_timediff_min_StartGame', 'BalloonBlast_sumtime_sec_StartRound', 'BalloonBlast_avgtime_sec_StartRound', 'BalloonBlast_sdtime_StartRound', 'BalloonBlast_count_PopBalloon', 'BalloonBlast_missed_PopBalloon', 'BalloonBlast_passed_PopBalloon', 'BalloonBlast_sumtime_sec_StartTutorial', 'BalloonBlast_avgtime_sec_StartTutorial', 'BalloonBlast_sdtime_StartTutorial', 'BalloonBlast_count_StartTutorial', 'MakeASplash_count_MouseDown', 'MakeASplash_sumtime_sec_DragPiece', 'MakeASplash_avgtime_sec_DragPiece', 'MakeASplash_sdtime_DragPiece', 'MakeASplash_count_DragPiece', 'MakeASplash_count_DropPieceInvalid', 'MakeASplash_timediff_min_StartGame', 'MakeASplash_sumtime_sec_StartRound', 'MakeASplash_avgtime_sec_StartRound', 'MakeASplash_sdtime_StartRound', 'MakeASplash_count_StartRound', '